----------------------------------------------------------------
FATAL AND INJURY COLUMNS
----------------------------------------------------------------
----------------------------------------------------------------

In [ ]:
!pip install plotly.express

import plotly.express as px
import pandas as pd

In [3]:
df = pd.read_excel('GSAF5.xls')
df.head()

,Date,Year,Type,Country,State,Location,Activity,Name,Sex,Age,...,Species,Source,pdf,href formula,href,Case Number,Case Number.1,original order,Unnamed: 21,Unnamed: 22
0,14th October,2025.0,Unprovoked,Columbia,"Bolivar, del Isolate",Catagena Province,Swimming with sharks,Male child,M,14,...,Nurse shark,Kevin McMurray Trackingsharks.com Andy Currie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,11th October,2025.0,Unprovoked,Australia,Queensland,Cook Esplanade Thursday Island,Fishing/swimming,Samuel Nai,M,14,...,Tiger or Bull shark,Kevin McMurray Trackingsharks.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,7th October,2025.0,Unprovoked,Australia,South Australia,Kangaroo Island,Surfing,Lee Berryman,M,50+,...,Bronze whaler?,Kevin McMurray Trackingsharks.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,29th September,2025.0,Unprovoked,USA,Off California,Catalina Island,Swimming,Christopher Murray,M,54,...,unknown 1.2m shark,Todd Smith: Kevin McMurray Trackingsharks.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,27th September,2025.0,Provoked,Costa Rica,NaN,Cocos Islands,Diving-Tagging sharks,Dr. Mauricio Hoyos,M,48,...,Tiger shark 4m,Todd Smith: Kevin McMurray Trackingsharks.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df['Injury'].value_counts() # Must clean Injury in to 3 answears only, 'yes', no' and 'unknown'.
# storing all the values that are not 'yes' or 'no' in 'unknown', e.g. '2017 -> unknown'.

Injury
FATAL                                                                                                     863
Foot bitten                                                                                               100
Survived                                                                                                   97
No injury                                                                                                  85
Leg bitten                                                                                                 81
                                                                                                         ... 
"Lost leg"                                                                                                  1
FATAL, body not recovered but shark was caught with the man's loincloth in its gut shortly afterwards.      1
FATAL, leg stripped of flesh                                                                                1
FAT

In [7]:
# Fatal Y/N column cleaning

df['Fatal Y/N'] = df['Fatal Y/N'].astype(str).str.strip().str.upper()

df['Fatal Y/N'] = df['Fatal Y/N'].replace({
    'N ': 'N',
    ' N': 'N',
    'UNKNOWN': 'Unknown',  
    'F': 'Unknown',
    'Y ': 'Y',
    'M': 'Unknown', 
    '2017': 'Unknown',
    'Y x 2': 'Unknown', # Y x 2 is ambiguous, so we set it to Unknown
    '0': 'Unknown'
})

df.loc[~df['Fatal Y/N'].isin(['Y', 'N']), 'Fatal Y/N'] = 'Unknown' # Makink sure only 'Y', 'N', and 'Unknown' remain

In [ ]:
df['Fatal Y/N'].value_counts(dropna=False) # We achieved less then 10% of unknowns in Fatal Y/N column

Fatal Y/N
N          4924
Y          1483
Unknown     643
Name: count, dtype: int64

In [ ]:
# Injury column cleaning and classification
# Replacing all types of injuries with these new classes we create from the coresponding values 
# we had before in Injury column:

df['Injury'] = df['Injury'].fillna('Unknown').astype(str).str.lower().str.strip()


def classify_injury(text):
    text = str(text).lower().strip()
    
    if any(word in text for word in ['no injury', 'uninjured', 'unharmed', 'none']):
        return 'No Injury'
    
    elif any(word in text for word in [
        'fatal', 'died', 'death', 'deceased',
        'human remains', 'drown', 'presumed dead', 'remains of the men'
    ]):
        return 'Fatal Wounds'
    
    elif any(word in text for word in [
        'minor', 'scratch', 'abrasion', 'bruise', 'small', 'superficial',
        'cut', 'scrape', 'nipped', 'grazed', 'bitten but not serious',
        'bumped', 'knocked', 'struck', 'collision', 'impact', 'puncture',
        'non-life-threatening'
    ]):
        return 'Minor Wounds'
    
    elif any(word in text for word in [ 
        'amputation', 'severe', 'major', 'deep', 'critical', 'multiple',
        'serious', 'massive', 'wound', 'bite', 'bitten', 'injury to',
        'injuries to', 'injured', 'injuries', 'damage', 'torn',
        'arm severed', 'leg severed', 'lacerat', 'tissue loss', 'survived', 'escaped'
    ]):
        return 'Severe Wounds' 
    
    else:
        return 'Unknown'

df['Injury_Class'] = df['Injury'].apply(classify_injury) # Creating a new column with the injury classification

df['Injury_Class'] = df['Injury_Class'].str.title() # Standardizing the format


In [ ]:
df['Injury_Class'].value_counts() # Achieved less then 10% of unknowns in Injury_Class column

Injury_Class
Severe Wounds    3264
Fatal Wounds     1621
No Injury         884
Minor Wounds      816
Unknown           465
Name: count, dtype: int64

In [ ]:
# Injury Class Distribution Analysis

# Count the injury classes
injury_counts = df['Injury_Class'].value_counts().reset_index()
injury_counts.columns = ['Injury_Class', 'Count']


fig = px.bar(injury_counts,
             x='Injury_Class',
             y='Count',
             color='Count',
             color_continuous_scale='Reds',
             title='Injury Class Distribution',)
fig.update_traces(textposition='outside')  # move labels above bars
fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide')

fig.show()
# Show the chart



In [ ]:
# We can see most reports are Severe wounds (~3.2k), followed by Fatal wounds (~1.6k) and about ~1.8k were minor and/or no wounds at all.
# This shows that more then 50% of the reports resulted in severe or fatal wounds.

------------------------------------------------------------------------------------

In [ ]:
# Fatal Cases Analysis
injury_counts = df['Fatal Y/N'].value_counts().reset_index()
injury_counts.columns = ['Fatal Y/N', 'Count']


fig = px.bar(injury_counts,
             x='Fatal Y/N',
             y='Count',
             color='Count',
             color_continuous_scale='Reds',
             title='Fatal Cases',)
fig.update_traces(textposition='outside')  # move labels above bars
fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide')

fig.show()

In [14]:
# With Fatal Cases we can say that about 75% of the cases were not fatal
# while about 15% were fatal and about 10% are unknown.
# Altough this seems good, we must be alert that about 3k non fatal cases from the 5k total, were also reported as severe wounds.

--------------------------------------
---------------------------------------


Fetched External Data for Tourism: